In [1]:
import torch, pickle, time, os, random
import numpy as np
import matplotlib.pyplot as plt
import torch_geometric as tg
from torch_geometric.loader import DataLoader
# accelerate huggingface to GPU
if torch.cuda.is_available():
    from accelerate import Accelerator
    accelerator = Accelerator()
    device = accelerator.device

torch.manual_seed(42)
random.seed(42)

In [2]:
os.listdir('../../../../scratch/gpfs/cj1223/GraphStorage/')

['medium_beforeafter',
 'medium_redshift_50_all',
 'test_onlyhmass_smass',
 'medium_redshift_80_all',
 'vlarge_all_final',
 'test_cut',
 'medium_all_q_diffbeforeafter',
 'vlarge_redshift_85_all',
 'medium_all_smass',
 'medium_onlyhmass_smass',
 'medium_scaleonly_smass',
 'vlarge_redshift_50_all',
 'test_beforeafter',
 'test_all_smass',
 'medium_redshift_15_all',
 'transformers',
 'medium_noinfonoedge_smass',
 'medium_onlyedge_smass',
 'medium_all_variance',
 'medium_redshift_1_all',
 'medium_all_q',
 'medium_redshift_10_all',
 'medium_all_final',
 'vlarge_redshift_95_all',
 'test_variance',
 'small_all_q_variancehalomass',
 'medium_all_residual',
 'medium_all',
 'test',
 'vlarge_all_robust_smass',
 'test_final',
 'vlarge_redshift_99_all',
 'vlarge_redshift_99.99_all',
 'vlarge_redshift_75_all',
 'med_all_residual',
 'medium_redshift_25_all',
 'medium_redshift_0.1_all',
 'vlarge_all_smass',
 'one_all_q',
 'test_redshift_scan',
 'medium_all_finalhalo',
 'test_dev',
 '105lim_all_smass',
 

In [3]:
# case='medium_hmassonly_smass'
# case='medium_scaleonly_smass'
### this didn't work as well as I initially thought, must have done something wrong, but still better than base
# case='medium_onlyedge_smass' 
# case='medium_noinfo_smass' 
case='vlarge_all_final'

data=pickle.load(open(f'../../../../scratch/gpfs/cj1223/GraphStorage/{case}/data.pkl', 'rb'))

In [4]:
len(np.array([data[0].y])), len(data)

(1, 108694)

In [5]:
import torch.nn.functional as F
from torch.nn import Linear, BatchNorm1d, LayerNorm
from torch_geometric.nn import SAGEConv, global_mean_pool, norm, global_max_pool, global_add_pool


class GCN(torch.nn.Module):
    def __init__(self, hidden_channels, nlin=3):
        super(GCN, self).__init__()
        
        self.conv1 = SAGEConv(data[0].num_node_features, hidden_channels) ##use meta-layer
        ## https://pytorch-geometric.readthedocs.io/en/latest/modules/nn.html#torch_geometric.nn.meta.MetaLayer
        self.conv2 = SAGEConv(hidden_channels, hidden_channels)
        self.conv3 = SAGEConv(hidden_channels, hidden_channels)
        self.conv4 = SAGEConv(hidden_channels, hidden_channels)
#         self.conv5 = SAGEConv(hidden_channels, hidden_channels)
        
        
        # Our final linear layer will define our output
        self.norm = LayerNorm(normalized_shape=hidden_channels) # layer_norm instead
        self.lin1 = Linear(hidden_channels, 1)
        
    def forward(self, x, edge_index, batch):
        # 1. Obtain node embeddings 
        x = self.conv1(x, edge_index)
        x = x.relu()
        x = self.conv2(x, edge_index)
        x = x.relu()
        x = self.conv3(x, edge_index)
        x = x.relu()
        x = self.conv4(x, edge_index)
        x = x.relu()
#         x = self.conv5(x, edge_index)
#         x = x.relu()
        x = global_add_pool(x, batch)  ## Miles says use sumpool

        x = self.lin1(self.norm(x))
#         x=self.lin1(x)
        return x
    
model = GCN(hidden_channels=32)
next(model.parameters()).is_cuda ##check number one

False

In [6]:
criterion = torch.nn.MSELoss()
n_epochs=50
n_trials=1
batch_size=128
split=0.8
test_data=data[int(len(data)*split):]
train_data=data[:int(len(data)*split)]

In [7]:
trains, tests, scatter = [], [], []
yss, preds=[],[]
for _ in range(n_trials):
    model = GCN(hidden_channels=32)
    train_loader=DataLoader(train_data, batch_size=batch_size, shuffle=1)
    test_loader=DataLoader(test_data, batch_size=batch_size, shuffle=0)    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.003)
    _, _, test_loader = accelerator.prepare(model, optimizer, test_loader)
    model, optimizer, train_loader = accelerator.prepare(model, optimizer, train_loader)
    print('GPU ', next(model.parameters()).is_cuda)
    # Initialize our train function
    def train():
        model.train()

        for data in train_loader:  
            out = model(data.x, data.edge_index, data.batch)  
            loss = criterion(out, data.y.view(-1,1)) 
#             loss.backward()
            accelerator.backward(loss)
            optimizer.step() 
            optimizer.zero_grad() 
     # test function
    def test(loader):
        model.eval()

        correct = 0
        for dat in loader: 
            out = model(dat.x, dat.edge_index, dat.batch) 
    #         print(out)
            correct += (torch.square(out - dat.y.view(-1,1))).sum() 
        return correct / len(loader.dataset) 
    tr_acc, te_acc=[],[]
    start=time.time()
    for epoch in range(n_epochs):
        train()

        if (epoch+1)%2==0:
            train_acc = test(train_loader).cpu().detach().numpy()
            test_acc = test(test_loader).cpu().detach().numpy()
            tr_acc.append(np.sqrt(train_acc))
            te_acc.append(np.sqrt(test_acc))
            print(f'Epoch: {epoch+1:03d}, Train scatter: {np.sqrt(train_acc):.4f}, Test scatter: {np.sqrt(test_acc):.4f}')
    stop=time.time()
    spent=stop-start
    tests.append(te_acc)
    trains.append(tr_acc)
    print(f"{spent:.2f} seconds spent training, {spent/n_epochs:.3f} seconds per epoch. Processed {len(data)*split*n_epochs/spent:.0f} trees per second")
    ys, pred=[],[]
    def test(loader):
        model.eval()

        correct = 0
        for dat in loader: 
            out = model(dat.x, dat.edge_index, dat.batch) 
            pred.append(out.view(1,-1).cpu().detach().numpy())
            ys.append(np.array(dat.y.cpu())) 
    test(test_loader)
    ys=np.hstack(ys)
    pred=np.hstack(pred)[0]
    scatter.append(np.std(ys-pred))
    yss.append(ys)
    preds.append(pred)

GPU  True
Epoch: 002, Train scatter: 0.1801, Test scatter: 0.1794
Epoch: 004, Train scatter: 0.2049, Test scatter: 0.1930
Epoch: 006, Train scatter: 0.1560, Test scatter: 0.1492
Epoch: 008, Train scatter: 0.1598, Test scatter: 0.1544
Epoch: 010, Train scatter: 0.1610, Test scatter: 0.1565
Epoch: 012, Train scatter: 0.1583, Test scatter: 0.1509
Epoch: 014, Train scatter: 0.1528, Test scatter: 0.1472
Epoch: 016, Train scatter: 0.1521, Test scatter: 0.1471
Epoch: 018, Train scatter: 0.1552, Test scatter: 0.1502
Epoch: 020, Train scatter: 0.1531, Test scatter: 0.1498
Epoch: 022, Train scatter: 0.1515, Test scatter: 0.1464
Epoch: 024, Train scatter: 0.1510, Test scatter: 0.1464
Epoch: 026, Train scatter: 0.1546, Test scatter: 0.1496
Epoch: 028, Train scatter: 0.1652, Test scatter: 0.1628
Epoch: 030, Train scatter: 0.1578, Test scatter: 0.1521
Epoch: 032, Train scatter: 0.1520, Test scatter: 0.1470
Epoch: 034, Train scatter: 0.1497, Test scatter: 0.1454
Epoch: 036, Train scatter: 0.1505, Tes

In [ ]:
np.mean(scatter), np.std(scatter)

In [ ]:
np.min(tests)

In [ ]:
plt.plot(tr_acc, label='train loss')
plt.plot(te_acc, label='test loss')
plt.xscale('log')
plt.yscale('log')
plt.legend()

In [ ]:
ys, pred=[],[]  
def test(loader):
    model.eval()
 
    correct = 0
    for dat in loader: 
        out = model(dat.x, dat.edge_index, dat.batch) 
        pred.append(out.view(1,-1).cpu().detach().numpy())
        ys.append(np.array(dat.y.cpu())) 
test(test_loader)
# test(train_loader)
ys=np.hstack(ys)
pred=np.hstack(pred)[0]

In [ ]:
fig, ax= plt.subplots(2, figsize=(9,12))
bins=50
ax[0].hist(ys, bins=bins, range=(np.percentile(np.hstack([ys,pred]), [1,99])), label='true',  histtype='step')
ax[0].hist(pred, bins=bins, range=(np.percentile(np.hstack([ys,pred]), [1,99])), label='pred', histtype='step')
ax[1].hist((ys-pred), bins=bins,range=(np.percentile(ys-pred, [1,99])),  histtype='step', label='residuals');
ax[0].legend()
ax[1].legend()
ax[0].set(title='Predicted/true stellar mass', xlabel='log10(M_stellar)', ylabel='N')
ax[1].set(title=f'Residuals, scatter is {np.std(ys-pred):.3f}',xlabel='log10(M_stellar)_true-log10(M_stellar)_predicted', ylabel='N')

In [ ]:
fig, ax =plt.subplots(figsize=(12,9))
ax.plot(ys,pred, 'ro')
ax.plot([min(ys),max(ys)],[min(ys),max(ys)], 'k--', label='Perfect correspondance')
ax.set(xlabel='true',ylabel='predicted', title='True/predicted corr')
ax.legend()

In [ ]:
fig, ax =plt.subplots(figsize=(12,9))
h = ax.hist2d(ys,pred, bins=50, range=[np.percentile(ys, [1,99]),np.percentile(pred, [1,99])])
fig.colorbar(h[3], ax=ax)
ax.plot([min(ys),max(ys)],[min(ys),max(ys)], 'k--', label='Perfect correspondance')
ax.set(xlabel='true',ylabel='predicted', title='True/predicted corr')
ax.legend()

In [ ]:
Mh=[]
for x in xes:
    Mh.append(x[0,3])
plt.plot(Mh, pred, 'o', markersize=1) 

In [ ]:
x,y=[], []
for d in data:
    x.append(d.x[0,3].numpy())
    y.append(d.y.numpy())    

In [ ]:
plt.plot(x, y, 'o', markersize=1) 

In [ ]:
## remember to validate that test/val/train are actually comparable
## create toy problem (predict variance of all halo masses eg, verifying that in takes into account edges)
## same, predict average of the difference of post-merge halo and pre merger halos

## try different pools

In [ ]:
xte, yte, lte = [], [], []
for d in test_data:
    xte.append(d.x.numpy()[0,3])
    yte.append(d.y.numpy())    
    lte.append(np.log10(len(d.x.numpy()))   ) 
yte=np.vstack(yte)

In [ ]:
xtr, ytr, ltr = [], [], []
for d in train_data:
    xtr.append(d.x.numpy()[0,3])
    ytr.append(d.y.numpy())    
    ltr.append(np.log10(len(d.x.numpy()))   ) 
ytr=np.vstack(ytr)

In [ ]:
bins=20
fig, ax = plt.subplots(3, figsize=(12,12))
ax[0].hist(xtr, bins=bins, histtype='step', density=1, label='train')
ax[0].hist(xte, bins=bins, histtype='step', density=1, label='test')
ax[0].set(xlabel='Final halo mass')

ax[1].hist(ytr, bins=bins, histtype='step', density=1, label='train')
ax[1].hist(yte, bins=bins, histtype='step', density=1, label='test')
ax[1].set(xlabel='Stellar mass')

ax[2].hist(ltr, bins=bins, histtype='step', density=1, label='train')
ax[2].hist(lte, bins=bins, histtype='step', density=1, label='test');
ax[2].set(xlabel='log10(Tree length)')

for a in ax:
    a.set(yscale='log')
    a.legend()